In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
base_url = 'https://letterboxd.com'
user = 'scoutog'
len_user_name = len(user) + 1
diary_urls = []
ratings = [5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1, .5, 'none']
df = pd.DataFrame()

In [ ]:
def get_entry_details(dataframe, soup_entry, rating_num):
    
    data = []
    
    # Extract film titles and ratings
    for film in film_list:
        # Get film title from 'alt' attribute of 'img' tag
        film_title = film.find('img')['alt']

        # Get rating from the 'span' tag within 'p' tag with class 'poster-viewingdata'
        if rating_num == 'none':
            film_rating = ""
            
        else:
            film_rating = film.find('p', class_='poster-viewingdata').find('span', class_='rating').text

        # Appending the extracted information to the data list
        data.append({
            'Title': film_title,
            'Rating_txt': film_rating,
            'Rating_num' : rating_num
        })

    # Creating a DataFrame from the data list
    entry_df = pd.DataFrame(data)
    
    df = pd.concat([dataframe, entry_df]).reset_index(drop=True)
    
    return df

In [ ]:
df = pd.DataFrame()

# Loop through ratings
for rating in tqdm(ratings):
    page = 1
    
    # Loop through pages
    while True:
        r = requests.get(f'{base_url}/{user}/films/rated/{rating}/page/{page}/')
        soup = BeautifulSoup(r.content, "html.parser")
        
        if soup.find('ul', class_='poster-list') == None:
            break
        elif len(soup.find('ul', class_='poster-list')) <= 1:
            break
        else:
            film_list = soup.find('ul', class_='poster-list').find_all('li', class_='poster-container')
            df = get_entry_details(df, film_list, rating)
            page += 1

In [ ]:
df

In [ ]:
pair_df = pd.DataFrame({'pair_key': list(zip(df['Title'], df['Rating_num']))})

In [ ]:
pair_df.pair_key.unique()